https://github.com/SeleniumHQ/docker-selenium

docker run -d -p 4444:4444 -p 7900:7900 --shm-size="2g" selenium/standalone-firefox:latest

Point your WebDriver tests to http://localhost:4444
To see what is happening inside the container, head to http://localhost:7900 (password is secret)


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from re import sub as re_sub
from collections import namedtuple
from decimal import Decimal


In [ ]:
options = FirefoxOptions()
driver = webdriver.Remote(
    command_executor='http://172.17.0.3:4444',
    options=options
)

In [ ]:
driver.get('https://www.mikroe.com/click')
menu=driver.find_element(By.XPATH, "(//*[contains(@class, 'tree dynamized')])")
cb_menu=menu.find_element(By.PARTIAL_LINK_TEXT,"Click Boards")

In [ ]:
ProductHierarchy = namedtuple("ProductHierarchy", "category subcategory href")
Products = namedtuple("Products", "name code subcategory url description unit_price")

In [ ]:
product_hierarchy = []

category_elements = cb_menu.find_elements(By.XPATH, './../ul/*')
for element in category_elements:
    category = element.find_element(By.TAG_NAME, 'a').get_attribute("textContent")
    category = re_sub(r"\([^()]*\)", "", category).strip()
    if category in ["Click Bundles", "Click Shields", "Legacy"]:
        continue
    subcategory_elements = element.find_elements(By.XPATH, 'ul/*')

    for element in subcategory_elements:
        subcategory = element.find_element(By.TAG_NAME, 'a').get_attribute("textContent")
        subcategory = re_sub(r"\([^()]*\)", "", subcategory).strip()
        href = element.find_element(By.TAG_NAME, 'a').get_attribute("href")
        
        product_hierarchy.append(ProductHierarchy(category, subcategory, href))


In [ ]:
products = []
subcategory_count = len(product_hierarchy)
for _ in product_hierarchy:
    driver.get(str(_.href))
    subcategory = _.subcategory

    product_elements=driver.find_elements(By.XPATH, "(//*[contains(@class, 'product-container')])")
    product_count = len(product_elements)
    print(f"{subcategory_count} - {subcategory} has {product_count} products")
    for element in product_elements:
        code = element.find_element(By.CSS_SELECTOR, "div.content_price span.product-reference").get_attribute("textContent")
        name = element.find_element(By.CLASS_NAME, "product-name").get_attribute("textContent")
        url = element.find_element(By.CLASS_NAME, "product-name").get_attribute("href")
        description = element.find_element(By.CLASS_NAME, "product-desc").get_attribute("textContent")
        unit_price = element.find_element(By.CLASS_NAME, "me-product-price").get_attribute("textContent")
        unit_price = Decimal(re_sub(r'[^\d.]', '', unit_price))
        
        products.append(Products(name, code, subcategory, url, description, unit_price))
    
    subcategory_count = subcategory_count-1

In [ ]:
len(product_hierarchy)

In [ ]:
# things = driver.find_elements(By.XPATH, "//div[@class='af_pl_wrapper']")
# things = driver.find_elements(By.CSS_SELECTOR, "li.ajax_block_product")
# things = driver.find_elements(By.CSS_SELECTOR, "div.product-container.clearfix")
# things = driver.find_elements(By.XPATH, "(//*[contains(@class, 'product-container')])")
# x = things[0].find_element(By.CSS_SELECTOR, "p.product-desc")

In [ ]:
driver.quit()

In [ ]:
products